In [1]:
from PIL import Image, ImageDraw
import torch
from torch.utils.data import DataLoader
from torchvision import transforms, datasets
import numpy as np
import pandas as pd
from time import time
import sys, os
import glob
from tqdm.auto import tqdm

from facenet_pytorch.models.mtcnn import MTCNN, prewhiten
from facenet_pytorch.models.inception_resnet_v1 import InceptionResnetV1, get_torch_home
from facenet_pytorch.models.utils.detect_face import extract_face

In [2]:
def get_image(path, trans):
    img = Image.open(path)
    img = trans(img)
    return img

In [3]:
trans = transforms.Compose([
    transforms.Resize(512)
])

trans_cropped = transforms.Compose([
    np.float32,
    transforms.ToTensor(),
    prewhiten
])

In [4]:
dataset = datasets.ImageFolder('dataset/lfw', transform=trans)
dataset.idx_to_class = {k: v for v, k in dataset.class_to_idx.items()}
loader = DataLoader(dataset, collate_fn=lambda x: x[0])

In [5]:
mtcnn = MTCNN(device=torch.device('cuda:0'))

In [6]:
resnet = InceptionResnetV1(pretrained='casia-webface').eval().cuda()

In [8]:
total_item = len(dataset)
names = []
aligned = []
embs = []
for img, idx in tqdm(loader):
    name = dataset.idx_to_class[idx]
    # start = time()
    img_align = mtcnn(img)#, save_path = "data/aligned/{}/{}.png".format(name, str(idx)))
    # print('MTCNN time: {:6f} seconds'.format(time() - start))
    if img_align is not None:
        names.append(name)
#         aligned.append(img_align)
        embs.append(resnet(img_align.unsqueeze(0).cuda()).cpu().detach().numpy())

embs = torch.stack(embs)

TypeError: expected Tensor as element 0 in argument 0, but got numpy.ndarray

In [13]:
df = pd.DataFrame({"name": names,
                  "emb": embs})

In [14]:
df.head()

,name,emb
0,AJ_Cook,"[[-0.015349547, 0.041209806, 0.04593148, -0.04..."
1,AJ_Lamas,"[[-0.10907699, -0.009377478, 0.051964134, -0.0..."
2,Aaron_Eckhart,"[[-0.056046043, 0.060716446, -0.020927187, -0...."
3,Aaron_Guiel,"[[-0.034311973, 0.07039009, 0.005534758, -0.10..."
4,Aaron_Patterson,"[[0.018119339, 0.012262784, -0.022838615, -0.0..."


In [15]:
df.to_csv("embeddings.csv", index=False)

In [ ]:
10

In [7]:
arr = aligned.numpy()
np.save("aligned", arr)

In [2]:
aligned = np.load('aligned.npy')
aligned = torch.from_numpy(aligned)

In [3]:
import torch
from torch.utils import data

class Dataset(data.Dataset):
  'Characterizes a dataset for PyTorch'
  def __init__(self, aligned_pics):
        'Initialization'
        self.aligned_pics = aligned_pics

  def __len__(self):
        'Denotes the total number of samples'
        return len(self.aligned_pics)

  def __getitem__(self, index):
        'Generates one sample of data'
        # Select sample
        aligned_pic = self.aligned_pics[index]

        return aligned_pic

In [4]:
dataset = Dataset(aligned)
data_loader = DataLoader(dataset, batch_size=64, shuffle=False)

In [6]:
next(iter(data_loader)).shape

torch.Size([64, 3, 160, 160])

In [7]:
# resnet = InceptionResnetV1(pretrained='casia-webface').eval().cuda()
# # aligned = aligned.cuda()
# start = time()
# embs = resnet(next(iter(data_loader)))
# print('\nResnet time: {:6f} seconds\n'.format(time() - start))

# # dists = [[(emb - e).norm().item() for e in embs] for emb in embs]

# # print('\nOutput:')
# # print(pd.DataFrame(dists, columns=names, index=names))


Resnet time: 2.007630 seconds



In [8]:
resnet = InceptionResnetV1(pretrained='casia-webface').eval().cuda()

torch.Size([64, 512])